# Input of txt-file -> transferring to dataFrame

In [1]:
import numpy as np
import pandas as pd

import matplotlib.patches as patches

# from google.colab import files

In [45]:
# Read a CSV file into a DataFrame
# path_annotations = '/content/annotations.txt'

path_annotations = '/Users/myself/Pictures/Kestrix Bootcamp Project/Training_Data_Kestrix/obj_Train_data/DJI_20230504173742_0003_V.txt'
path_annotations_2 = '/Users/myself/Pictures/Kestrix Bootcamp Project/Training_Data_Kestrix/obj_Train_data/DJI_20230504173806_0037_V.txt'

column_names = ["class", "x_center", "y_center", "width", "height"]

# Read the file with a space delimiter and assign the column names
df = pd.read_csv(path_annotations_2, names=column_names, delimiter=' ')

# Display the DataFrame
print(df)

df.shape

    class  x_center  y_center     width    height
0       0  0.716679  0.110162  0.055942  0.141463
1       0  0.973158  0.143825  0.053685  0.115950
2       0  0.988533  0.031653  0.022935  0.063307
3       0  0.956321  0.030792  0.049492  0.061583
4       0  0.908910  0.030102  0.052320  0.059517
5       0  0.811746  0.022523  0.054393  0.045047
6       0  0.760325  0.021317  0.050250  0.042633
7       0  0.709546  0.018905  0.046892  0.037810
8       1  0.474663  0.083660  0.020275  0.051880
9       1  0.011637  0.672783  0.023275  0.022367
10      1  0.007971  0.807723  0.015943  0.022367
11      0  0.974425  0.597852  0.051150  0.159830
12      0  0.675997  0.892187  0.061695  0.108440
13      0  0.738173  0.884870  0.057855  0.137793
14      1  0.304744  0.411660  0.027598  0.030593
15      0  0.705824  0.433038  0.055942  0.141463


(16, 5)

# Function: **splitting_into_compartments**

# Function: **luc_coordinates**

In [3]:
def luc_coordinates(num_width_comp, num_height_comp, comp_size):
  ''' A function that takes as an input:
  width_comp = number of compartments for the width (e.g. 8)
  height_comp = number of compartments for the height (e.g. 6)
  comp_size = size of each compartment (e.g. 640)
  and returns the coordinates of the left-upper-corner (luc) of
  each compartment
  '''

  # Height and width of the input image
  width = 4000
  height = 3000

  # Step size is the size of the compartments unique pixels horizontally and vertically
  step_size_horiz = width / num_width_comp
  step_size_vert = height / num_height_comp

  # Defining how much overlap there will be between each compartment vertically and horizontally
  overlap_horiz = int(((comp_size * num_width_comp) - width)/num_width_comp)
  overlap_vert = int(((comp_size * num_height_comp) - height)/num_height_comp)

  #creating a list with all the x_coordinates for the compartments (8)
  x_coordinates = [0]
  [x_coordinates.append(int(ele * step_size_horiz)) for ele in range(1, num_width_comp)]

  # creating a list with all the y_coordinates for the compartments (6)
  y_coordinates = [0]
  [y_coordinates.append(int(ele * step_size_vert)) for ele in range(1, num_height_comp)]


  # Creating a dictionary with the coordinates
  coordinates_dict = {}
  b = 0 # no inherent important meaning

  for ele in range (0, num_height_comp):
    for i in range (0, num_width_comp):
      a = [x_coordinates[i], y_coordinates[ele]]
      b += 1
      coordinates_dict[f'cor_{b}'] = a

  return coordinates_dict

In [4]:
def splitting_into_compartments(tensor):
  '''
  Takes as an input the Tensor that represents the
  whole image that is supposed to be split into compartments.
  The tensor should have the shape (4140, 3140, 3)
  '''

  # Calling the function 'luc_coordinates' and saving the resulting dictionary of coordinates in a variable
  coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

  slize_size = 640

  # Turning the coordinates into a list
  coordinates_list = [value for key, value in coordinates_dict.items()]

  # creating a for loop for getting the correct slizing integers and putting them into a dictionary
  slicing_dict = {}

  for i in range (0, 48):
    slice_1, slice_2 = coordinates_list[i][0], coordinates_list[i][1]
    slice_1_a = slice_1
    slice_1_b = slice_1_a + slize_size
    slice_2_a = slice_2
    slice_2_b = slice_2_a + slize_size
    slicing_dict[i] = [slice_1_a, slice_1_b, slice_2_a, slice_2_b]

  # slicing the input-tensor to get (48) of shape (640, 640, 3)
  list_of_tensors = [tensor[slicing_dict[ele][0]:slicing_dict[ele][1],
                                 slicing_dict[ele][2]:slicing_dict[ele][3],

                                 ] for ele in range (0, 48)]

  #optional, if needed:
  compartment_tensors = np.array(list_of_tensors)

  return compartment_tensors

# Function: **slicing_dictionary**

In [5]:
def slicing_dictionary(tensor):

  # Calling the function 'luc_coordinates' and saving the resulting dictionary of coordinates in a variable
  # coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

  slize_size = 640

  # Turning the coordinates into a list
  coordinates_list = [value for key, value in coordinates_dict.items()]

  # creating a for loop for getting the correct slizing integers and putting them into a dictionary
  slicing_dict = {}

  for i in range (0, 48):
    slice_1, slice_2 = coordinates_list[i][0], coordinates_list[i][1]
    slice_1_a = slice_1
    slice_1_b = slice_1_a + slize_size
    slice_2_a = slice_2
    slice_2_b = slice_2_a + slize_size
    slicing_dict[i] = [slice_1_a, slice_1_b, slice_2_a, slice_2_b]

  return slicing_dict

# Function: getting_bounding_boxes

In [6]:
def getting_bounding_boxes(width_img, height_img, comp_size, bb):
  '''
  Requirement: Having access to the annotations for one image in
  a pd.DataFrame
  Inputs:
  width_img = Width of the input image (right now 4000)
  height_img = Height of the input image (right now 3000)
  comp_size = size we want for each compartment (right now 640 (x 640))
  bb = bounding box
  Outputs:
  We calculate: x_min, y_min, x_max, y_max which can then be used
  to get the coordinates of each corner of the bounding box
  '''
  # Defining the width and height of the compartment
  width_comp = comp_size
  height_comp = comp_size

  # getting the x_center, y_center, width, height from the first bounding box
  x_center = df.iloc[bb]['x_center']                  # = 0.785663
  y_center = df.iloc[bb]['y_center']                  # = 0.776272
  first_width = df.iloc[bb]['width']                  # = 0.05003
  first_height = df.iloc[bb]['height']                # = 0.109817
  class_object = int(df.iloc[bb]['class'])            # = 0 ('car')

  #MAYBE splitting what's above and creating a for-loop for all the annotations

  # Transferring to the actual width and height of the image
  x_center = round(x_center * width_img)            # = 3143
  y_center = round(y_center * height_img)           # = 2329
  first_width = round(first_width * width_img)        # = 200
  first_height = round(first_height * height_img)     # = 329

  # Calculating the 4 cornerpoints, for which we need 4 numbers: x_min, x_max, y_min, y_max
  # (Adding 70 for each because of the padding of zeros around the original image)
  x_min = int(x_center - (first_width / 2)) + 70  # left = 3043
  x_max = int(x_center + (first_width / 2)) + 70 # right = 3243
  y_min = int(y_center - (first_height / 2))+ 70 # top = 2164
  y_max = int(y_center + (first_height / 2))+ 70 # bottom = 2493

  return x_min, y_min, x_max, y_max, class_object


# Function: checking_bounding_box



In [7]:
def checking_bounding_box(compartment_num):
  '''
  Input to this function is the number of the compartment for which it
  should be tested whether the bounding box is inside or not
  And the output of function: getting_bounding_boxes
  With this we want to take one row of the txt.file (pd.DataFrame)
  that represents one bounding box in the original image
  Output of the function:
  True if some part of the bounding box lies in the compartment
  False if the bounding box doesn't lie in the compartment!
  '''

  # If one of x_min, x_max AND one of y_min, y_max in the compartment -> True else False
  if ((x_min in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] + 1)
    or x_max in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] +1))
    and
     ((y_min in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1)
    or y_max in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1)))):
    tmp_var = True
  else:
    tmp_var = False


#[slicing_dict[0][0]:slicing_dict[0][1],
#slicing_dict[0][2]:slicing_dict[0][3]
  return tmp_var

# Function: checking_for_9_cases

In [8]:
def checking_for_9_cases():
  '''
  ONLY ACTIVATE IF function: checking_bounding_box == TRUE
  Because then we know: Some part of that bounding box lies in
  this compartment!
  We want to
  '''

  # Checking for the next cases (9!)

  # Setting x_min to its value if it is in the range - otherwise setting it to be at the edge (x_min = 0!)
  if x_min in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] + 1):
    x_min_tmp = x_min
  else:
    x_min_tmp = slicing_dict[compartment_num][0]

  # Setting x_max to its value if it is in the range - otherwise setting it to be at the edge (x_max = 1!)
  if x_max in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] + 1):
    x_max_tmp = x_max
  else:
    x_max_tmp = slicing_dict[compartment_num][1]

  # Doing the same for y_min and y_max!
  if y_min in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1):
    y_min_tmp = y_min
  else:
    y_min_tmp = slicing_dict[compartment_num][2]

  if y_max in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1):
    y_max_tmp = y_max
  else:
    y_max_tmp = slicing_dict[compartment_num][3]

  # Returning the 4 coordinates in the XYXY class - format
  return x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp

# Function: to_absolute_coordinates_xyxy


In [9]:
def to_absolute_coordinates_xyxy():
  '''
  Calculates the absolute coordinates of a bounding box
  for a single compartment
  Output: absolute coordinates in form (xyxy = x_min, y_min, x_max, y_max)
  '''
  abs_x_min = x_min_tmp - slicing_dict[compartment_num][0]
  # print(slicing_dict[compartment_num][0])
  abs_x_max = x_max_tmp - slicing_dict[compartment_num][0]
  abs_y_min = y_min_tmp - slicing_dict[compartment_num][2]
  # print(slicing_dict[compartment_num][2])
  abs_y_max = y_max_tmp - slicing_dict[compartment_num][2]

  return abs_x_min, abs_y_min, abs_x_max, abs_y_max

# Pipeline (1)

**Input**

In [46]:
# Needed Variables

#for 'luc_coordinates'
num_width_comp = 8
num_height_comp = 6
comp_size = 640

#for 'slicing_dict'
tensor = np.ones((4140, 3140, 3))

#for 'getting_bounding_boxes'
width_img = 4000
height_img = 3000
#comp_size

**1. Pipe**

In [47]:
# Pipeline from Input-tensor to slicing_dict + cp_bb

def pipeline_input_to_slicing_dict():

  # Calling the function 'luc_coordinates'
  coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

  #Calling the function 'slicing_dictionary'
  slicing_dict = slicing_dictionary(tensor)

  # Image-level: Getting the bounding box coordinates ONCE for the whole image
  corner_points_of_bounding_boxes_original_image_with_padding = {}

  for i in range(0, df.shape[0]):

    bb = i
    #function: getting_bounding_boxes
    x_min, y_min, x_max, y_max, class_object = getting_bounding_boxes(width_img, height_img, comp_size, bb)
    corner_points_of_bounding_boxes_original_image_with_padding[bb] = [x_min, y_min, x_max, y_max, class_object]

  cp_bb = corner_points_of_bounding_boxes_original_image_with_padding  # = dict
  cp_bb

  return slicing_dict, cp_bb


**2. Pipe**

In [48]:
def preparing_for_for_loops():

    # Create an empty DataFrame with the specified columns
    columns = ['class', 'x_min', 'y_min', 'x_max', 'y_max']
    data_frame = pd.DataFrame(columns=columns)
    data_frame

    # Running this once, so that I have a dictionary full of names for the dataframes that come out of my loops
    dict_df_comps = {}

    for i in range (0, 48):
        dict_df_comps[i] = f'df_comp_{i}'

    return data_frame, dict_df_comps

**1. und 2. Pipe executed**

In [49]:
slicing_dict, cp_bb = pipeline_input_to_slicing_dict()
# slicing_dict, cp_bb

data_frame, dict_df_comps = preparing_for_for_loops()
# data_frame, dict_df_comps

**Global Run (Function doesn't work)**

In [59]:
# For Loop for each compartment (compartment-level)
for key, value in slicing_dict.items():
  #print(key)
  #print(value)
  compartment_num = key
  print(f'compartment', key)
  dict_df_comps[key] = data_frame.copy()

  # For Loop for each bounding box
  for k, v in cp_bb.items():

    # calling function: checking_bounding_box (Output = True or False)
    x_min = v[0]
    y_min = v[1]
    x_max = v[2]
    y_max = v[3]
    class_object = v[4]
    #print(x_min)
    #print(y_min)
    #print(x_max)
    #print(y_max)
    if checking_bounding_box(compartment_num):
      print('True')
      #calling the function: checking_for_9_cases
      x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases()
      # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
      abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy()
      # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

      #Adding those to a dataframe with the original class
      list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
      dict_df_comps[key].loc[len(dict_df_comps[key])] = list_ready
    # else:
      # print('False')
      # continue or cut the else
  print(dict_df_comps[key])

  #Output is the dict_df_comps

compartment 0
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 1
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 2
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 3
True
   class  x_min  y_min  x_max  y_max
0      1    428    243    509    399
compartment 4
True
   class  x_min  y_min  x_max  y_max
0      1      0    243      9    399
compartment 5
True
True
True
   class  x_min  y_min  x_max  y_max
0      0    325    188    549    612
1      0    510     70    640    198
2      0    314     70    502    183
compartment 6
True
True
True
True
True
   class  x_min  y_min  x_max  y_max
0      0      0    188     49    612
1      0    601     70    640    249
2      0    208     70    426    205
3      0     10     70    211    198
4      0      0     70      2    183
compartment 7
True
True
True
True
   class  x_min  y_min  x_max  y_max
0      0    355    327    570    640
1      0    4

**Getting the desired Output**

(= A dictionary with Key = number of compartment, Value = DataFrame with annotations for this compartment)

In [60]:
dict_df_comps[15]

,class,x_min,y_min,x_max,y_max
0,0,355,0,570,175


**Constructing the exporting-function with a for-loop**

In [72]:
for key, _ in dict_df_comps.items():
    # print(key, value)
    # Save DataFrame to .txt file
    dict_df_comps[key].to_csv(f'/Users/myself/Kestrix_Project/data/txt_files_48/annot_for_comp_{key}.txt', sep=' ',index=False, index_label = False, header=False)



# Starting to test for the Output (manually)

In [12]:
# Inputs
num_width_comp = 8
num_height_comp = 6
comp_size = 640

tensor = np.ones((4140, 3140, 3))

# Calling the function 'luc_coordinates'
coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

# Calling the function 'slicing_dictionary'
slicing_dict = slicing_dictionary(tensor)

coordinates_dict, slicing_dict

({'cor_1': [0, 0],
  'cor_2': [500, 0],
  'cor_3': [1000, 0],
  'cor_4': [1500, 0],
  'cor_5': [2000, 0],
  'cor_6': [2500, 0],
  'cor_7': [3000, 0],
  'cor_8': [3500, 0],
  'cor_9': [0, 500],
  'cor_10': [500, 500],
  'cor_11': [1000, 500],
  'cor_12': [1500, 500],
  'cor_13': [2000, 500],
  'cor_14': [2500, 500],
  'cor_15': [3000, 500],
  'cor_16': [3500, 500],
  'cor_17': [0, 1000],
  'cor_18': [500, 1000],
  'cor_19': [1000, 1000],
  'cor_20': [1500, 1000],
  'cor_21': [2000, 1000],
  'cor_22': [2500, 1000],
  'cor_23': [3000, 1000],
  'cor_24': [3500, 1000],
  'cor_25': [0, 1500],
  'cor_26': [500, 1500],
  'cor_27': [1000, 1500],
  'cor_28': [1500, 1500],
  'cor_29': [2000, 1500],
  'cor_30': [2500, 1500],
  'cor_31': [3000, 1500],
  'cor_32': [3500, 1500],
  'cor_33': [0, 2000],
  'cor_34': [500, 2000],
  'cor_35': [1000, 2000],
  'cor_36': [1500, 2000],
  'cor_37': [2000, 2000],
  'cor_38': [2500, 2000],
  'cor_39': [3000, 2000],
  'cor_40': [3500, 2000],
  'cor_41': [0, 2500]

In [13]:
# Image-level: Getting the bounding box coordinates ONCE for the whole image
width_img = 4000
height_img = 3000
comp_size = 640

corner_points_of_bounding_boxes_original_image_with_padding = {}

for i in range(0, df.shape[0]):

  bb = i
  #function: getting_bounding_boxes
  x_min, y_min, x_max, y_max, class_object = getting_bounding_boxes(width_img, height_img, comp_size, bb)
  corner_points_of_bounding_boxes_original_image_with_padding[f'bb_{bb}'] = [x_min, y_min, x_max, y_max, class_object]

cp_bb = corner_points_of_bounding_boxes_original_image_with_padding  # = dict
cp_bb

{'bb_0': [3113, 2234, 3313, 2563, 0],
 'bb_1': [1045, 1164, 1233, 1302, 0],
 'bb_2': [3398, 2199, 3591, 2531, 0],
 'bb_3': [3854, 2135, 4045, 2439, 0],
 'bb_4': [2321, 2258, 2512, 2613, 0],
 'bb_5': [2635, 2221, 2827, 2590, 0]}

In [52]:
def new_func(dict_df_comps):
    '''
    docstring
    '''

    # For Loop for each compartment (compartment-level)
    for key, value in slicing_dict.items():
        #print(key)
        #print(value)
        compartment_num = key
        print(key, f'compartment')
        dict_df_comps[key] = data_frame.copy()

        # For Loop for each bounding box
        for k, v in cp_bb.items():

            # calling function: checking_bounding_box (Output = True or False)
            x_min = v[0]
            y_min = v[1]
            x_max = v[2]
            y_max = v[3]
            class_object = v[4]
            #print(x_min)
            #print(y_min)
            #print(x_max)
            #print(y_max)
            if checking_bounding_box(compartment_num):
                print('True')
                #calling the function: checking_for_9_cases
                x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases()
                # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
                abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy()
                # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

                #Adding those to a dataframe with the original class
                list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
                dict_df_comps[key].loc[k] = list_ready
                # else:
                # print('False')
                # continue or cut the else
        # print(dict_df_comps[key])
    return dict_df_comps

        #Output is the dict_df_comps

In [53]:
data_frame, dict_df_comps = preparing_for_for_loops()

dont_care = new_func(dict_df_comps)
dont_care

0 compartment
1 compartment
2 compartment
3 compartment
4 compartment
5 compartment
6 compartment
7 compartment
8 compartment
9 compartment
10 compartment
11 compartment
12 compartment
13 compartment
14 compartment
15 compartment
16 compartment
17 compartment
18 compartment
19 compartment
20 compartment
21 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
22 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
23 compartment
24 compartment
25 compartment
26 compartment
27 compartment
28 compartment
29 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
30 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
31 compartment
32 compartment
33 compartment
34 compartment
35 compartment
36 compartment
37 compartment
38 compartment
39 compartment
40 compartment
41 compartment
42 compartment
43 compartment
44 compartment
45 compartment


{0: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 1: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 2: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 3: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 4: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 5: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 6: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 7: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 8: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 9: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 10: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 11: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 12: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_

In [14]:
# Create an empty DataFrame with the specified columns
columns = ['class', 'x_min', 'y_min', 'x_max', 'y_max']
data_frame = pd.DataFrame(columns=columns)
data_frame

# Running this once, so that I have a dictionary full of names for the dataframes that come out of my loops
dict_df_comps = {}

for i in range (0, 48):
  dict_df_comps[i] = f'df_comp_{i}'

# For Loop for each compartment (compartment-level)
for key, value in slicing_dict.items():
  #print(key)
  #print(value)
  compartment_num = key
  print(key, f'compartment')
  dict_df_comps[key] = data_frame.copy()

  # For Loop for each bounding box
  for k, v in cp_bb.items():

    # calling function: checking_bounding_box (Output = True or False)
    x_min = v[0]
    y_min = v[1]
    x_max = v[2]
    y_max = v[3]
    class_object = v[4]
    #print(x_min)
    #print(y_min)
    #print(x_max)
    #print(y_max)
    if checking_bounding_box(compartment_num):
      print('True')
      #calling the function: checking_for_9_cases
      x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases()
      # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
      abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy()
      # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

      #Adding those to a dataframe with the original class
      list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
      dict_df_comps[key].loc[0] = list_ready
    # else:
      # print('False')
      # continue or cut the else
  print(dict_df_comps[key])

  #Output is the dict_df_comps

0 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
1 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
2 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
3 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
4 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
5 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
6 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
7 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
8 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
9 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
10 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
11 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_

,class,x_min,y_min,x_max,y_max
0,0,135,221,327,590
